### Prérequis: Avoir fait le tp Transformer.ipynb

# Génération de texte
Félicitation d'avoir fini le tp sur les transformers (ou pas). Ici, on va s'amuser à générer du texte avec un transformer (ou presque).

## Idée
Je ne sais pas si vous avez remarqué, mais pour de la génération de texte, il n'y pas de phrase à encoder, contrairement à la traduction de texte où la phrase à encoder est la phrase à traduire.

On va donc utiliser juste la partie decodeur du transformer. Mais si vous vous souvenez, il y a la couche Cross Attention dans la partie decodeur, qui n'est pas nécessaire ici car on n'a pas de phrase à encoder. On va donc utiliser un decoduer sans la Cross Attention.

![decoderonly](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/decoderonly.png)

Bref, l'idée est la suivante: on va entraîner notre modèle à compléter des phrases.

Exemple:

Entrée: ["\<start>", "Je", "suis", "un", "chat", "<end>"]

Sortie: ["Je", "suis", "un", "chat", "\<end>"]

Le modèle va alors:
- Prédire "Je" à partir de "\<start>"
- Prédire "suis" à partir de "\<start> Je"
- Prédire "un" à partir de "\<start> Je suis"
- etc...

Puis, pendant l'inference, on va juste lui donner "\<start>" et il va compléter la phrase jusqu'à "\<end>".

Si vous vous rappelez, la sortie de notre modèle est des vecteurs de probabilité et dans le tp précédent, on a utilisé l'argmax de ces vecteurs de probabilité pour avoir le mot prédit.

Mais si ici on prend l'argmax du vecteur de probabilité à chaque fois, on va avoir juste avoir la même phrase à chaque fois qu'on génère du texte.

Donc on va utiliser un autre moyen pour générer du texte: le sampling.

Le sampling consiste à prendre un mot en fonction de sa probabilité. Par exemple, si le mot "chat" a une probabilité de 0.8, on va le prendre 80% du temps.

Donc avec les vecteurs de probabilité que notre modèle nous donne, on va prendre un mot en fonction de sa probabilité.

Avant de sampler, on va diviser les vecteurs (avant le softmax) par un facteur qu'on appelle la température. La température est un hyperparamètre qui va déterminer la diversité du texte généré. Plus la température est grande, plus le texte généré sera diversifié. Plus la température est petite, plus le texte généré sera similaire à l'entrée, mais plus robuste à l'erreur, car avec une température trop grande, les probabilités des mots vont être trop proches et on risque d'avoir des mots qui n'ont pas de sens.

Remarque: C'est exactement ce que fait ChatGPT. Il est aussi un decoder-only transformer mais il a 175 milliards de paramètres, un peu plus gros que le modèle qu'on va faire ici.

## Préparation des données
On va utiliser le dataset WikiText-2.

Il faut tout d'abord installer torchtext et portalocker.

In [ ]:
!pip install torchtext portalocker

Je créer le `vocab` et le padding pour vous. Vous aurez `x_train` déjà préparé, et `pad_train` qui est un masque pour le padding.

In [ ]:
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import AG_NEWS
import torch

tokenizer = get_tokenizer('basic_english')
train_iter = AG_NEWS(split='train')
# remove label for language modeling task
train_iter = map(lambda x: x[1], train_iter)

vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=["<unk>", "<pad>", "<start>", "<end>"])
vocab.set_default_index(vocab["<unk>"])

In [ ]:
x_train = []
max_len = 500
for sentence in train_iter:
    x_train.append(torch.tensor(vocab(["<start>"] + tokenizer(sentence)[:500] + ["<end>"])))

x_train = torch.nn.utils.rnn.pad_sequence(x_train, batch_first=True, padding_value=vocab["<pad>"])
pad_train = (x_train != vocab["<pad>"]).bool()

## A vous de jouer
Créez un modèle et entraînez-le.

## Génération de texte

Après 20 minutes d'entraînement sur colab, j'ai obtenu: "<start> a sol forces and is a song on five years , which is seized at the £ b . she is a cyclone churches when she is a day of the refurbishment . him to shiva and financial to the giant and a legislation . <end>"